In [1]:
import warnings
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import re
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def initialize_df():
    '''Create the necessary dataframes for extracting features
    from the files.
    '''
    global features, maping_description, insurance_features, ads_features

    features = {
        'name': None,
        'model': None,
        'price': None,
        'color': None,
        'fuel_type': None,
        'origin_car': None,
        'car_license': None,
        'lime_type': None,
        'glass': None,
        'motor_power': None,
        'car_speedometer': None,
        'passengers': None,
        'payment_method': None,
        'displayed': None,
        'ex_owners': None,
        'additional_info': None,
        'insurance_third': None,
        'supplementary_body': None,
        'total': None,
        'ads_status': None,
        'ads_start_data': None,
        'ads_end_data': None,
        }
    maping_description = {
        "لون السيارة": 'color',
        "نوع الوقود": 'fuel_type',
        "أصل السيارة": 'origin_car',
        "عداد السيارة": 'car_speedometer',
        "أصحاب سابقون": 'ex_owners',
        "رخصة السيارة": 'car_license',
        "نوع الجير": 'lime_type',
        "الزجاج": 'glass',
        "قوة الماتور": 'motor_power',
        "عدد الركاب": 'passengers',
        "وسيلة الدفع": 'payment_method',
        "معروضة": 'displayed'
    }
    insurance_features = ['insurance_third', 'supplementary_body', 'total']
    ads_features = ['ads_status', 'ads_start_data', 'ads_end_data']

In [3]:
def get_model(model_element):
    """Extracting model year features from file to features dictionary.

    Args:
        model_element (bs4.element.Tag): contains first 'h5' element
            with no css class.
    """
    for row in model_element:
        subitem = str(row.text).split()
        for item in subitem:
            if(item.isdigit()):
                features['model'] = item
                break

In [4]:
def get_price(price_element):
    """Extracting price feature from file to features dictionary.
    Args:
        price (bs4.element.Tag): contain 'h5' tag with no css class
    """
    price = str(price_element.text)
    for subitem in price.split():
        if(subitem.isdigit()):
            price = subitem
            break

    features['price'] = int(price)

In [5]:
def get_description(description_element):
    """Extracting description features from file to features dictionary.
    it has 12 features: 'color', 'fuel_type', 'origin_car',
    'car_license', 'lime_type','glass', 'motor_power', 'car_speedometer',
    'passengers', 'payment_method','displayed', 'ex_owners',

    Args:
        description_element (bs4.element.Tag): contain 'table' element
            with css class 'list_ads'
    """

    for arabic_feature in maping_description.keys():
        description = description_element.find(text=arabic_feature)
        features[maping_description[arabic_feature]] = \
            None if description is None \
            else description.next_element.get_text()

In [6]:
def get_insurance(insurance_element):
    """Extracting insurance features from file to features dictionary.
    it has 3 features: 'insurance_third','supplementary_body' and 'total'

    Args:
        insurance_element (bs4.element.ResultSet): contain all 'td' element
            with no css class nor css colspan from the first div element
            has css class 'row'
    """
    count = 0
    for row in insurance_element:
        subitem = str(row.text).split()
        for item in subitem:
            if(item.isdigit()):
                features[insurance_features[count]] = item
                count += 1

In [7]:
def get_additional(additional_element):
    """Extracting additional feature from file to features dictionary.

    Args:
        additional_element (bs4.element.Tag): contain 'td' element with
            css class 'list-additions'
    """
    additional_value = []
    additional_element = additional_element.findAll('li')

    for row in additional_element:
        value = ''
        value += row.text
        additional_value.append(value)
    additional_value = ','.join(map(str, additional_value))

    features['additional_info'] = additional_value

In [34]:
def get_post_info(post_info_element):
    """Extracting post information features from file to features dictionary.
    it has 3 features: 'ads_status', 'ads_start_data' and 'ads_end_data'

    Args:
        post_info_element (bs4.element.ResultSet): contains the third 'table'
            element with css class 'create_post'
    """
    post_info_element = str(post_info_element.text).split()

    date_with_str = [post_info_element[5], post_info_element[8]]

    features['ads_status'] = post_info_element[2]

    for item in date_with_str:
        date = []
        for subitem in item.split('-'):
            if subitem.isdigit():
                date.append(str(subitem)) 
            else:
                number = ''
                for char in list(subitem):
                    if char.isdigit():
                        number += str(char)
        date.append(number)
        date = '-'.join(date)

In [45]:
# Create dataframe to add the data collected
data = pd.DataFrame()

for path in Path('data/').glob('*.txt'):
    initialize_df()
    with path.open(encoding='utf-8') as file_path:
        # create BeautifulSoup object for file by html parser
        soup = BeautifulSoup(file_path, "html.parser")

        # Extracting name value to features dictionary
        features['name'] = soup.find('h3', class_=None).text

        # Extracting model value to features dictionary
        get_model(soup.find('h5', class_=None))

        # Extracting price value to features dictionary
        get_price(soup.find('h5', class_='post-price'))

        # Extracting description values to features dictionary
        get_description(soup.find('table', class_='list_ads'))

        # Extracting insurance values to features dictionary
        get_insurance(soup.find('div', class_='row').findAll('td',
                      attrs={'class': None, 'colspan': None}))

        # Extracting additional value to features dictionary
        get_additional(soup.find('td', class_='list-additions'))

        # Extracting post information values to features dictionary
        get_post_info(soup.findAll('table', class_='create_post')[-1])

        # Convert the dictionary to series
        sample = pd.Series(features)

        # Concating the sample series to the dataframe
        data = pd.concat((data, sample), axis=1, ignore_index=True)

In [48]:
# Transpose dataframe
data = data.T

In [49]:
data

,name,model,price,color,fuel_type,origin_car,car_license,lime_type,glass,motor_power,...,displayed,ex_owners,additional_info,insurance_third,supplementary_body,drive,total,ads_status,ads_start_data,ads_end_data
0,كيا اوبتيما,2014,100000,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,...,للبيع فقط,يد اولى,"مُكيّف,إغلاق مركزي,جهاز إنذار,مسجل CD,فتحة سقف...",1340,1751,None,3090,منتهي,None,None
1,كيا سورينتو,2007,60000,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2500,...,للبيع أو التبديل,2,"مُكيّف,إغلاق مركزي,جهاز إنذار,مسجل CD,فتحة سقف...",1690,1050,None,2740,منتهي,None,None
2,هونداي افانتي,2006,43500,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600,...,للبيع فقط,None,"مُكيّف,إغلاق مركزي,جهاز إنذار,مسجل CD,جنطات مغ...",1340,1000,None,2340,منتهي,None,None
3,فيات 127,1982,5500,بيج,بنزين,خصوصي,فلسطينية,عادي,يدوي,906,...,للبيع فقط,00,"مُكيّف,مسجل CD",467,500,None,967,منتهي,None,None
4,بيجو 208,2014,54000,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1200,...,للبيع فقط,None,"مُكيّف,إغلاق مركزي,جهاز إنذار,جنطات مغنيسيوم,و...",1035,1000,None,2035,منتهي,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859,كيا مورننغ,2015,45000,فيراني,كهرباء,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1000,...,للبيع فقط,2,"مُكيّف,إغلاق مركزي,جهاز إنذار,جنطات مغنيسيوم,و...",935,1000,None,1935,تيم,None,None
6860,فورد ترانزيت,2002,48000,أبيض,ديزل,عمومي,فلسطينية,عادي,الكتروني,2400,...,للبيع فقط,5,,845,500,None,1345,البزار,None,None
6861,بيجو بارتنر,2018,87000,فضي,ديزل,خصوصي,فلسطينية,عادي,الكتروني,1600,...,للبيع فقط,يد صفر,"مُكيّف,إغلاق مركزي,جهاز إنذار,مسجل CD,وسادة حم...",1340,1523,None,2863,سميح,None,None
6862,كيا سورينتو,2017,126000,بترولي,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2200,...,للبيع فقط,ثانيه,"مُكيّف,إغلاق مركزي,جهاز إنذار,مسجل CD,فتحة سقف...",1690,2205,None,3895,منتهي,None,None


In [50]:
data.to_csv('data/data.csv')